In [1]:
# # Import necessary modules
# import sys
# import torch
# sys.path.append('/Users/zhenzhili/MASTERTHESIS/#Expert_System_Design/')

# from src.dpcepsrc.train import train_model
# from examples.MNIST.coins.mnist import MNIST_Net, MNIST_Classifier, test_MNIST, neural_predicate
# from src.dpcepsrc.data_loader import load
# from src.dpcepsrc.optimizer import SGD
# from src.dpcepsrc.model import Model
# from src.dpcepsrc.network import Network

# from problog.logic import Var
# from problog.parser import PrologParser
# from problog.program import ExtendedPrologFactory
# model_training = False
# origin_path = "/Users/zhenzhili/MASTERTHESIS/#Expert_System_Design/examples/MNIST/coins/" 
# model_path = "models/"
# rule_path = "rules/"
# origin_data_path = "data/origin/"
# bias_data_path = "data/bias/"



# def test_MNIST():
#     # Load the same ProbLog string/program that defines the model's structure
#     in_train_path = origin_path+origin_data_path+'in_test_data.txt'
#     coin_basic_path = origin_path+rule_path+'coin_basic.pl'
#     model_coin_path = origin_path+model_path+'model_iter_003890.mdl'
#     # queries_for_test = origin_path+"queries_for_test.txt"
#     origin_queries_path = origin_path+origin_data_path+"init_test_data.txt"
#     bias_queries_path = origin_path+bias_data_path+"init_test_data.txt"

#     prolog_string = ""
#     for str_file in [coin_basic_path, in_train_path]:
#         with open (str_file) as f:
#             prolog_string += f.read()
#             prolog_string += "\n"

#     network1 = MNIST_Net(N=2)
#     net1 = Network(network1, 'mnist_net1', neural_predicate)
#     net1.optimizer = torch.optim.Adam(network1.parameters(), lr=0.001)

#     network2 = MNIST_Net(N=2)
#     net2 = Network(network2, 'mnist_net2', neural_predicate)
#     net2.optimizer = torch.optim.Adam(network2.parameters(), lr=0.001)
#     # Create a new model instance with the same structure as the original
#     model = Model(prolog_string, [net1, net2], caching=False)
#     model.load_state(model_coin_path)

#     # Parse a string query into a Term: parseString, parseFile
#     # with open (queries_for_test) as query_f:
#     #         query_string = query_f.read()
#     with open (origin_queries_path) as ori_init_f:
#             ori_init_string = ori_init_f.read()
            
#     queries = PrologParser(ExtendedPrologFactory()).parseString(ori_init_string) # list

#     count = 0
#     for query in queries:
#         _, arg2 = query.args
#         query_not = query(Var("sequence=A"), arg2)
#         # print(query)
#         result = model.solve(query_not)
#         single_result = []
#         for q, (prob, _) in result.items():
#             binare_result = 1 if prob>0.5 else 0
#             correct = 1 if (str(q.args[0]) == f"sequence={binare_result}") else 0
#             count += correct
#             print(count)
#             single_result.append((q.args[0],format(prob,".2f")))
#     print("accuracy:", count / len(queries))

# test_MNIST()

# Test Biased Data:

In [2]:
import os
import sys
from mnist import MNIST_Net, MNIST_Classifier, test_MNIST, neural_predicate

sys.path.append('/Users/zhenzhili/MASTERTHESIS/#Expert_System_Design/')
from src.dpcepsrc.model import Model, Var
from src.dpcepsrc.data_loader import load
from src.dpcepsrc.network import Network
import torch

from examples.MNIST.coins.prob_ec_testing import test


def add_files_to(problog_files, problog_string):
    for problog_file in problog_files:
        with open(problog_file) as f:
            problog_string += f.read()
            problog_string += '\n\n'

    return problog_string


def test_MNIST(test_data, problog_files, problog_test_files, test_model):
    test_queries = load(test_data)

    problog_string = add_files_to(problog_files, '\n')
    problog_test_string = add_files_to(problog_test_files, problog_string)

    network1 = MNIST_Net(N=2)
    net1 = Network(network1, 'mnist_net1', neural_predicate)
    net1.optimizer = torch.optim.Adam(network1.parameters(), lr=0.001)
    network2 = MNIST_Net(N=2)
    net2 = Network(network2, 'mnist_net2', neural_predicate)
    net2.optimizer = torch.optim.Adam(network2.parameters(), lr=0.001)

    model_to_test = Model(problog_test_string, [net1,net2], caching=False)
    model_to_test.load_state(test_model)

    res = test(model_to_test, 
            test_queries, 
            test_functions={
                'mnist_net1': lambda *args, **kwargs: neural_predicate(*args, **kwargs, dataset='test'),
                'mnist_net2': lambda *args, **kwargs: neural_predicate(*args, **kwargs, dataset='test')
            },
          )
    return res

In [3]:
#===========#===========# bias_test #===========#===========#
bias_test = True
test_now = True
#===========#===========# orig test #===========#===========#

if bias_test: 
    data_path = "data/bias"
else:
    data_path = "data/origin"
rules_path = "rules"
model_path = "models"
if test_now:
    res = test_MNIST(
        # os.path.join(data_path,'detect_test_data.txt'),    # test_data: init_test_data.txt  detectEvent
        os.path.join(data_path,'result_test_data.txt'),    # test_data: init_test_data.txt  detectEvent

        # [os.path.join(rules_path,"coin_simple_event_format.pl")],
        [os.path.join(rules_path,"modify.pl")],

        [os.path.join(data_path,'in_test_data.txt')],    # problog_test_files: happensAt (put a list here)

        os.path.join(model_path,'model_iter_basic_detect.mdl'),
        # os.path.join(model_path,'model_iter_listall.mdl'),
    )

100%|██████████| 401/401 [00:07<00:00, 51.27it/s]

result
[[196   4]
 [  5 196]]
F1 result: [0.97755611 0.97755611]
Accuracy result: 0.9775561097256857


<pre>
Origin: detectEvent
[[199   1]
 [  6 203]]
F1 detectEvent: [0.98271605 0.98305085]
Accuracy detectEvent: 0.9828850855745721

Bias:  detectEvent
 [[142  63]
 [ 50 146]]
F1 detectEvent: [0.78365385 0.76683938]
Accuracy detectEvent: 0.7755610972568578

# Test Generating Rules:

In [4]:
def read_file(data_path, file_name, read_samples=False):
    file_path = os.path.join(data_path, file_name)
    with open(file_path) as f:
        if not read_samples:
            file_str = f.read()
        else:
            file_lines = f.readlines()[:5]
            file_str = "\n".join(file_lines)
    return file_str

In [5]:
bias_test = False

if bias_test: 
    data_path = "data/bias"
else:
    data_path = "data/origin"
rules_path = "rules"
model_path = "models"
context_str ="""
This code combines the outputs of two different neural networks (corresponding to event1 and event2 respectively), 
extracts information from the simultaneous events through logical rules, and determines the final event outcome based on the difference between the two event labels.
"""
rules_str = read_file(rules_path, "coin_basic_modify.pl")
facts_str = read_file(data_path, "in_test_data.txt",read_samples=True)
queries_str = read_file(data_path, "cp_test_data.txt",read_samples=True)
question_str = """
When the absolute value of the difference between the outputs of e1 and e2 is 2, the game output is 1, otherwise the game output is 0
"""
# question_str = """
# when the timestamp is between 50 and 250, if the event1 and event2 are classified as 1 and 3, the sequence should equals 0
# when the timestamp is between 100 and 300, if the event1 and event2 are classified as 0 and 3, the sequence should equals 1
# """

FileNotFoundError: [Errno 2] No such file or directory: 'data/origin/cp_test_data.txt'

In [ ]:
from src.functions import get_model_files, get_single_model_file_with_path, prolog_reasoning
if not test_now:
    from src.symbcot_groq import Groq_Reasoning_Graph_Baseline
    args = [
        context_str,
        rules_str,
        facts_str,
        queries_str,
        question_str,
        "deepseek-r1-distill-llama-70b", # model_name
        "", # stop_words
        "src/generated", # save_path
        500, # max_new_tokens
        False # use_inline_prompt?
    ]
    deepseek_problem_reduction = Groq_Reasoning_Graph_Baseline(*args)
    deepseek_problem_reduction.reasoning_graph_generation()

Translating...
model_name:  deepseek-r1-distill-llama-70b
save_directory_thinks: src/generated/thinks/2025-03-06
save_directory_models: src/generated/models/2025-03-06


In [ ]:
model_files = get_model_files("2025-02-07", "deepseek")

Found 0


In [ ]:
model = """
coin1(ID1, T) :-
    happensAt(X,Y,T), 
    event1(X, ID1).
coin2(ID2, T) :-
    happensAt(X,Y,T), 
    event2(Y, ID2).

outcome(1, ID1, ID2) :- (abs(ID1 - ID2) =:= 2).
outcome(0, ID1, ID2) :- (abs(ID1 - ID2) =\= 2).

detectEvent(sequence=EventID,T) :-
    coin1(ID1, T),
    coin2(ID2, T),
    outcome(EventID, ID1, ID2).

% =================%%  Following will be code generated by LLM  %%=================%
% ================= %% ================= %% ================= %% ================= %

detectEvent(sequence=0, T) :-
    % When timestamp is between 50 and 250, and event1=1, event2=3
    coin1(1, T),
    coin2(3, T),
    T >= 50,
    T =< 250.

detectEvent(sequence=1, T) :-
    % When timestamp is between 100 and 300, and event1=0, event2=3
    coin1(0, T),
    coin2(3, T),
    T >= 100,
    T =< 300.

"""

facts = """ 
happensAt(a,b,150).
event1(a,0).
event2(b,3).

query(detectEvent(X, 150)).
"""

prolog_reasoning(model+facts)

% ProbLog Inference Result：
detectEvent(sequence=0,150) = 1.0000
detectEvent(sequence=1,150) = 1.0000


In [ ]:
t1 = 1
t2 = "a"
str(f"{t1 and t2}")

'a'